# Coursera Capstone project
This project is for the IBM introduction to data science course. The project should used to apply the learned topics in  an own project.

Author: Sebastian Freidank

## Part 1 Welcome to the Capstone Project

In [1]:
import pandas as pd
import numpy as np

print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Part 2
## Segmenting and Clustering Neighborhoods in Toronto
1. Import table from website and convert to pandas dataframe.

In [2]:
import requests # library to handle requests
#from bs4 import BeautifulSoup
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url)

#print(results.text)

#soup = BeautifulSoup(results.text, 'html.parser')

#table = soup.find_all('table')[0] 
#df = pd.read_html(str(table))

dfs = pd.read_html(results.text, header=0)[0]
dfs

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


2. Clean table from not assigned borough and neighboorhoods:

In [3]:
dfs = dfs[dfs['Borough'] != "Not assigned"]
dfs

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


 Shape of dataframe:

In [4]:
dfs.shape

(103, 3)

## Get geo location of postal code
Load geo data vom downloaded csv file:

In [5]:
geo = pd.read_csv("Geospatial_Coordinates.csv")
geo

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Add geo data to postal codes:

In [6]:
tmp = dfs.copy()
#tmp.loc[tmp['Postal Code'] == geo['Postal Code'], 'Latitude'] = geo['Latitude']
for index in geo['Postal Code']:
    for index2 in tmp['Postal Code']:
        if index == index2:
            #print(f'Index: {index}, Index2: {index2}')
            #print(geo.loc[geo['Postal Code'] == index, 'Longitude'])
            tmp.loc[tmp['Postal Code'] == index2, 'Longitude'] = geo.loc[geo['Postal Code'] == index, 'Longitude'].values
            tmp.loc[tmp['Postal Code'] == index2, 'Latitude'] = geo.loc[geo['Postal Code'] == index, 'Latitude'].values
            #print(tmp.loc[tmp['Postal Code'] == index2, 'Longitude'])
            #print(tmp.loc[tmp['Postal Code'] == index2, 'Latitude'])
            
    #tmp.loc[tmp['Postal Code'] == index, 'Longitude'] = geo['Longitude']    
tmp.reset_index(drop=True, inplace=True)
tmp

,Postal Code,Borough,Neighbourhood,Longitude,Latitude
0,M3A,North York,Parkwoods,-79.329656,43.753259
1,M4A,North York,Victoria Village,-79.315572,43.725882
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",-79.360636,43.654260
3,M6A,North York,"Lawrence Manor, Lawrence Heights",-79.464763,43.718518
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",-79.389494,43.662301
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",-79.506944,43.653654
99,M4Y,Downtown Toronto,Church and Wellesley,-79.383160,43.665860
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",-79.321558,43.662744
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",-79.498509,43.636258


## Part 3 Visualization

In [19]:
tmp['Neighbourhood']

0                                              Parkwoods
1                                       Victoria Village
2                              Regent Park, Harbourfront
3                       Lawrence Manor, Lawrence Heights
4            Queen's Park, Ontario Provincial Government
                             ...                        
98         The Kingsway, Montgomery Road, Old Mill North
99                                  Church and Wellesley
100    Business reply mail Processing Centre, South C...
101    Old Mill South, King's Mill Park, Sunnylea, Hu...
102    Mimico NW, The Queensway West, South of Bloor,...
Name: Neighbourhood, Length: 103, dtype: object

In [7]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [24]:
latitude = 43.725950
longitude = -79.338393
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(tmp['Latitude'], tmp['Longitude'], tmp['Borough'], tmp['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto